# Amazon - Product Review Python Web Scraper

In this notebook we will be obtaining our desired data (comments associated with purchased products by users) by web scraping reviews from every protein product currently selled at Amazon's website.
Sample product being searched is 'whey Protein', which is a sports/dietary supplement 'powder'.

#### Import Python libraries

In [1]:
from bs4 import BeautifulSoup
from datetime import datetime
from decimal import Decimal
import pandas as pd
import re
import requests

#### Set up URL

In [2]:
search_query= "whey+protein"
base_url= "https://www.amazon.com/s?k="
url= base_url + search_query

#### Confirm if we are allowed to scrape ('200' means our request was successful)

In [3]:
header= {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36','referer':'https://www.amazon.com/s?k=whey+protein&crid=58MOJL49EL5O&sprefix=whey%2Caps%2C156&ref=nb_sb_ss_ts-doa-p_2_4'}
search_response= requests.get(url,headers=header)
search_response.status_code

200

In [ ]:
search_response.text

In [4]:
search_response.cookies

<RequestsCookieJar[Cookie(version=0, name='session-id', value='142-9645336-8822158', port=None, port_specified=False, domain='.amazon.com', domain_specified=True, domain_initial_dot=True, path='/', path_specified=True, secure=True, expires=1708577843, discard=False, comment=None, comment_url=None, rest={}, rfc2109=False), Cookie(version=0, name='session-id-time', value='2082787201l', port=None, port_specified=False, domain='.amazon.com', domain_specified=True, domain_initial_dot=True, path='/', path_specified=True, secure=True, expires=1708577843, discard=False, comment=None, comment_url=None, rest={}, rfc2109=False), Cookie(version=0, name='i18n-prefs', value='USD', port=None, port_specified=False, domain='.amazon.com', domain_specified=True, domain_initial_dot=True, path='/', path_specified=True, secure=False, expires=1708577843, discard=False, comment=None, comment_url=None, rest={}, rfc2109=False), Cookie(version=0, name='sp-cdn', value='"L5Z9:MX"', port=None, port_specified=False,

## Create function to get the content of the page of required query

In [5]:
cookie= {} # insert request cookies within{}
def getAmazonSearch(search_query):
    url= "https://www.amazon.com/s?k=" + search_query
    print(url)
    page= requests.get(url,headers= header)
    if page.status_code == 200:
        return page
    else:
        return "Error"

## Create function to get the contents of individual product pages using 'data-asin' number (unique identification number)

In [6]:
def searchAsin(asin):
    url= "https://www.amazon.com/dp/" + asin
    print(url)
    page= requests.get(url, cookies= cookie, headers= header)
    if page.status_code == 200:
        return page
    else:
        return "Error"

## Create function to pass on the link of 'see all reviews' and extract the content

In [7]:
def searchReviews(review_link):
    url= "https://www.amazon.com" + review_link
    print(url)
    page= requests.get(url, cookies= cookie, headers= header)
    if page.status_code == 200:
        return page
    else:
        return "Error"

## First page product of reviews' extraction

In [8]:
product_names= []
response= getAmazonSearch('whey+protein')
soup= BeautifulSoup(response.content)
for i in soup.findAll("span", {'class':'a-size-base-plus a-color-base a-text-normal'}): # the tag which is common for all the names of products
    product_names.append(i.text) #adding the product names to the list

https://www.amazon.com/s?k=whey+protein


In [9]:
product_names

['Optimum Nutrition Gold Standard 100% Whey Protein Powder, Double Rich Chocolate 2 Pound (Packaging May Vary)',
 'Optimum Nutrition Gold Standard 100% Whey Protein Powder, French Vanilla Creme, 2 Pound (Packaging May Vary)',
 'Body Fortress Super Advanced Whey Protein Powder, Chocolate, Immune Support (1), Vitamins C & D Plus Zinc, 1.78 lbs',
 'Optimum Nutrition Gold Standard 100% Whey Protein Powder, Extreme Milk Chocolate, 2 Pound (Packaging May Vary)',
 'Dymatize Elite 100% Whey Protein Powder, 25g Protein, 5.5g BCAAs & 2.7g L-Leucine, Quick Absorbing & Fast Digesting for Optimal Muscle Recovery, Gourmet, 5 Pound Vanilla 80 Ounce',
 "Body Fortress Whey Protein Powder, 60g Protein and 12g BCAA's (per 2 scoops), Chocolate, 5 Lb.",
 'Optimum Nutrition Gold Standard 100% Whey Protein Powder, Vanilla Ice Cream, 5 Pound (Packaging May Vary)',
 'Muscle Milk 100% Whey Protein Powder, Chocolate, 5 Pound, 66 Servings, 25g Protein, 2g Sugar, Low in Fat, NSF Certified for Sport, Energizing Sna

In [10]:
len(product_names)

51

## The next step is extracting data-asin numbers that are linked to that of product names

In [11]:
data_asin= []
response= getAmazonSearch('whey+protein')
soup= BeautifulSoup(response.content)
for i in soup.findAll("div",{'class':'sg-col-4-of-24 sg-col-4-of-12 s-result-item s-asin sg-col-4-of-16 AdHolder sg-col s-widget-spacing-small sg-col-4-of-20'}):
    data_asin.append(i['data-asin'])
for i in soup.findAll("div",{'class':'sg-col-4-of-24 sg-col-4-of-12 s-result-item s-asin sg-col-4-of-16 sg-col s-widget-spacing-small sg-col-4-of-20'}):
    data_asin.append(i['data-asin'])

https://www.amazon.com/s?k=whey+protein


In [12]:
response.status_code

200

In [13]:
data_asin

['B0BRT77ZK8',
 'B07C5TWDQB',
 'B09P3KR9W5',
 'B002DYIZH6',
 'B002DYIZHQ',
 'B0BJLBD427',
 'B002DYIZHG',
 'B00EACSHF4',
 'B000GISU1M',
 'B010OVRDM2',
 'B0106ZJ4R8',
 'B09SY9SD4D',
 'B08R5FTZ52',
 'B002QZN8JW',
 'B00ZFONQZW',
 'B01N7VLTW3',
 'B006E54GJG',
 'B000GIQT3I',
 'B008IC0Z8Y',
 'B099J2SWXV',
 'B00G4MA4B8',
 'B07PD797QL',
 'B07DJL1PJT',
 'B06Y5ZG66K',
 'B00CUDYY2U',
 'B000GIQSVG',
 'B099HZG89P',
 'B099HXNRKK',
 'B000GIURIQ',
 'B00E7IODXQ',
 'B002DYJ0FM',
 'B099J4JWFQ',
 'B000GIPJY8',
 'B07FKW8MP3',
 'B0015R3AH2',
 'B01JNUHA5A',
 'B0015AQL1Q',
 'B002DYJ02U',
 'B002N6F2UW',
 'B074JKZXLB',
 'B071JG5QYT',
 'B00BEOHFKO',
 'B002DYJ1C4',
 'B008JGI6TE',
 'B07PBXBGTP',
 'B002TG3QPO',
 'B002DYIZSA',
 'B00G6QHYZ6',
 'B07FL5NVJZ',
 'B002SG7NG8',
 'B01JUA770G']

In [14]:
len(data_asin)

51

### By passing the data-asin numbers, we can extract the 'see all reviews' link for each product in the page

In [15]:
link= []
for i in range(len(data_asin)):
    response= searchAsin(data_asin[i])
    soup= BeautifulSoup(response.content)
    for i in soup.findAll("a", {'data-hook':"see-all-reviews-link-foot"}):
        link.append(i['href'])

https://www.amazon.com/dp/B0BRT77ZK8
https://www.amazon.com/dp/B07C5TWDQB
https://www.amazon.com/dp/B09P3KR9W5
https://www.amazon.com/dp/B002DYIZH6
https://www.amazon.com/dp/B002DYIZHQ
https://www.amazon.com/dp/B0BJLBD427
https://www.amazon.com/dp/B002DYIZHG
https://www.amazon.com/dp/B00EACSHF4
https://www.amazon.com/dp/B000GISU1M
https://www.amazon.com/dp/B010OVRDM2
https://www.amazon.com/dp/B0106ZJ4R8
https://www.amazon.com/dp/B09SY9SD4D
https://www.amazon.com/dp/B08R5FTZ52
https://www.amazon.com/dp/B002QZN8JW
https://www.amazon.com/dp/B00ZFONQZW
https://www.amazon.com/dp/B01N7VLTW3
https://www.amazon.com/dp/B006E54GJG
https://www.amazon.com/dp/B000GIQT3I
https://www.amazon.com/dp/B008IC0Z8Y
https://www.amazon.com/dp/B099J2SWXV
https://www.amazon.com/dp/B00G4MA4B8
https://www.amazon.com/dp/B07PD797QL
https://www.amazon.com/dp/B07DJL1PJT
https://www.amazon.com/dp/B06Y5ZG66K
https://www.amazon.com/dp/B00CUDYY2U
https://www.amazon.com/dp/B000GIQSVG
https://www.amazon.com/dp/B099HZG89P
h

In [16]:
len(link)

90

### Now we have the 'see all review' links. Using this link along with a page number, we can extract the reviews in any number of pages for all the products

In [17]:
link

['/NatureWorks-HydroMATE-Electrolytes-Chocolate-Count/product-reviews/B0BRT77ZK8/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',
 '/My-Whey-Portable-Supplement-Keychain/product-reviews/B07C5TWDQB/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',
 '/YIMOOCH-1500W-Power-Polisher/product-reviews/B09P3KR9W5/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',
 '/OPTIMUM-NUTRITION-STANDARD-Protein-Chocolate/product-reviews/B002DYIZH6/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',
 '/OPTIMUM-NUTRITION-STANDARD-Protein-Chocolate/product-reviews/B002DYIZH6/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',
 '/OPTIMUM-NUTRITION-STANDARD-Protein-Vanilla/product-reviews/B002DYIZHQ/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',
 '/OPTIMUM-NUTRITION-STANDARD-Protein-Vanilla/product-reviews/B002DYIZHQ/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',
 '/Body-Fortress-Advanced-Chocolate-Vitamins/product-rev

In [18]:
len(link)

90

## We adjust a range get 10 pages worth of reviews from each and ALL whey protein products currently offered at Amazon. NOTE: this step can take time to load

In [19]:
reviews= []
link_per_comment= []
rating_score= []
rating_date= []
for j in range(len(link)):
    for k in range(1,11):
        response= searchReviews(link[j] + '&pageNumber=' + str(k))
        soup= BeautifulSoup(response.content)
        url_text= "https://www.amazon.com" + link[j] + '&pageNumber=' + str(k)
        #comment_results= soup.findAll("span",{'data-hook':"review-body"})
        #rating_results= soup.findAll("i",{'data-hook':"review-star-rating"})
        #dates_results= soup.findAll("span",{'data-hook':"review-date"})
        for i in soup.findAll("span",{'data-hook':"review-body"}):
            reviews.append(i.text)
            link_per_comment.append(url_text)
        for i in soup.findAll("i",{'data-hook':"review-star-rating"}):
            rating_score.append(i.text if i.text is not None else "None")
            #if i.text is not None:
            #    rating_score.append(i.text)
            #else:
            #    rating_score.append("None")
            #rating_score.append(i.text if i.text != None else "None")
        for i in soup.findAll("span",{'data-hook':"review-date"}):
            rating_date.append(i.text) 

https://www.amazon.com/NatureWorks-HydroMATE-Electrolytes-Chocolate-Count/product-reviews/B0BRT77ZK8/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=1
https://www.amazon.com/NatureWorks-HydroMATE-Electrolytes-Chocolate-Count/product-reviews/B0BRT77ZK8/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=2
https://www.amazon.com/NatureWorks-HydroMATE-Electrolytes-Chocolate-Count/product-reviews/B0BRT77ZK8/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=3
https://www.amazon.com/NatureWorks-HydroMATE-Electrolytes-Chocolate-Count/product-reviews/B0BRT77ZK8/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=4
https://www.amazon.com/NatureWorks-HydroMATE-Electrolytes-Chocolate-Count/product-reviews/B0BRT77ZK8/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=5
https://www.amazon.com/NatureWorks-HydroMATE-Electrolytes-Chocolate-Count/product-reviews/B0BRT77ZK8/ref=cm_cr_dp_d_show_all_

https://www.amazon.com/OPTIMUM-NUTRITION-STANDARD-Protein-Vanilla/product-reviews/B002DYIZHQ/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=1
https://www.amazon.com/OPTIMUM-NUTRITION-STANDARD-Protein-Vanilla/product-reviews/B002DYIZHQ/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=2
https://www.amazon.com/OPTIMUM-NUTRITION-STANDARD-Protein-Vanilla/product-reviews/B002DYIZHQ/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=3
https://www.amazon.com/OPTIMUM-NUTRITION-STANDARD-Protein-Vanilla/product-reviews/B002DYIZHQ/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=4
https://www.amazon.com/OPTIMUM-NUTRITION-STANDARD-Protein-Vanilla/product-reviews/B002DYIZHQ/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=5
https://www.amazon.com/OPTIMUM-NUTRITION-STANDARD-Protein-Vanilla/product-reviews/B002DYIZHQ/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=

https://www.amazon.com/Optimum-Nutrition-Standard-Chocolate-Packaging/product-reviews/B002DYIZHG/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=10
https://www.amazon.com/Optimum-Nutrition-Standard-Chocolate-Packaging/product-reviews/B002DYIZHG/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=1
https://www.amazon.com/Optimum-Nutrition-Standard-Chocolate-Packaging/product-reviews/B002DYIZHG/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=2
https://www.amazon.com/Optimum-Nutrition-Standard-Chocolate-Packaging/product-reviews/B002DYIZHG/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=3
https://www.amazon.com/Optimum-Nutrition-Standard-Chocolate-Packaging/product-reviews/B002DYIZHG/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=4
https://www.amazon.com/Optimum-Nutrition-Standard-Chocolate-Packaging/product-reviews/B002DYIZHG/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerTyp

https://www.amazon.com/Body-Fortress-Advanced-Protein-Chocolate/product-reviews/B010OVRDM2/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=9
https://www.amazon.com/Body-Fortress-Advanced-Protein-Chocolate/product-reviews/B010OVRDM2/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=10
https://www.amazon.com/Muscle-Milk-Protein-Powder-Vanilla/product-reviews/B0106ZJ4R8/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=1
https://www.amazon.com/Muscle-Milk-Protein-Powder-Vanilla/product-reviews/B0106ZJ4R8/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=2
https://www.amazon.com/Muscle-Milk-Protein-Powder-Vanilla/product-reviews/B0106ZJ4R8/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=3
https://www.amazon.com/Muscle-Milk-Protein-Powder-Vanilla/product-reviews/B0106ZJ4R8/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=4
https://www.amazon.com/Muscle-Mil

https://www.amazon.com/Protein-MuscleTech-Hormone-Non-GMO-Vanilla/product-reviews/B08R5FTZ52/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=10
https://www.amazon.com/OPTIMUM-NUTRITION-Platinum-Hydrowhey-Hydrolyzed/product-reviews/B002QZN8JW/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=1
https://www.amazon.com/OPTIMUM-NUTRITION-Platinum-Hydrowhey-Hydrolyzed/product-reviews/B002QZN8JW/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=2
https://www.amazon.com/OPTIMUM-NUTRITION-Platinum-Hydrowhey-Hydrolyzed/product-reviews/B002QZN8JW/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=3
https://www.amazon.com/OPTIMUM-NUTRITION-Platinum-Hydrowhey-Hydrolyzed/product-reviews/B002QZN8JW/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=4
https://www.amazon.com/OPTIMUM-NUTRITION-Platinum-Hydrowhey-Hydrolyzed/product-reviews/B002QZN8JW/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerTy

...

https://www.amazon.com/Levels-Vanilla-Grass-Protein-Undenatured/product-reviews/B071JG5QYT/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=1
https://www.amazon.com/Levels-Vanilla-Grass-Protein-Undenatured/product-reviews/B071JG5QYT/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=2
https://www.amazon.com/Levels-Vanilla-Grass-Protein-Undenatured/product-reviews/B071JG5QYT/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=3
https://www.amazon.com/Levels-Vanilla-Grass-Protein-Undenatured/product-reviews/B071JG5QYT/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=4
https://www.amazon.com/Levels-Vanilla-Grass-Protein-Undenatured/product-reviews/B071JG5QYT/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=5
https://www.amazon.com/Levels-Vanilla-Grass-Protein-Undenatured/product-reviews/B071JG5QYT/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=6
https://ww

https://www.amazon.com/BSN-SYNTHA-6-Protein-Micellar-Servings/product-reviews/B002DYJ1C4/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=1
https://www.amazon.com/BSN-SYNTHA-6-Protein-Micellar-Servings/product-reviews/B002DYJ1C4/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=2
https://www.amazon.com/BSN-SYNTHA-6-Protein-Micellar-Servings/product-reviews/B002DYJ1C4/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=3
https://www.amazon.com/BSN-SYNTHA-6-Protein-Micellar-Servings/product-reviews/B002DYJ1C4/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=4
https://www.amazon.com/BSN-SYNTHA-6-Protein-Micellar-Servings/product-reviews/B002DYJ1C4/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=5
https://www.amazon.com/BSN-SYNTHA-6-Protein-Micellar-Servings/product-reviews/B002DYJ1C4/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=6
https://www.amazon.com

https://www.amazon.com/Isopure-Protein-Friendly-Unflavored-Packaging/product-reviews/B002TG3QPO/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=1
https://www.amazon.com/Isopure-Protein-Friendly-Unflavored-Packaging/product-reviews/B002TG3QPO/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=2
https://www.amazon.com/Isopure-Protein-Friendly-Unflavored-Packaging/product-reviews/B002TG3QPO/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=3
https://www.amazon.com/Isopure-Protein-Friendly-Unflavored-Packaging/product-reviews/B002TG3QPO/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=4
https://www.amazon.com/Isopure-Protein-Friendly-Unflavored-Packaging/product-reviews/B002TG3QPO/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=5
https://www.amazon.com/Isopure-Protein-Friendly-Unflavored-Packaging/product-reviews/B002TG3QPO/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_r

https://www.amazon.com/Dymatize-Protein-Isolate-Gourmet-Vanilla/product-reviews/B00G6QHYZ6/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=10
https://www.amazon.com/Optimum-Nutrition-Standard-Chocolate-Hazelnut/product-reviews/B07FL5NVJZ/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=1
https://www.amazon.com/Optimum-Nutrition-Standard-Chocolate-Hazelnut/product-reviews/B07FL5NVJZ/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=2
https://www.amazon.com/Optimum-Nutrition-Standard-Chocolate-Hazelnut/product-reviews/B07FL5NVJZ/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=3
https://www.amazon.com/Optimum-Nutrition-Standard-Chocolate-Hazelnut/product-reviews/B07FL5NVJZ/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=4
https://www.amazon.com/Optimum-Nutrition-Standard-Chocolate-Hazelnut/product-reviews/B07FL5NVJZ/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_revie

https://www.amazon.com/MuscleTech-NitroTech-Protein-Isolate-Peptides/product-reviews/B01JUA770G/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=9
https://www.amazon.com/MuscleTech-NitroTech-Protein-Isolate-Peptides/product-reviews/B01JUA770G/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=10
https://www.amazon.com/MuscleTech-NitroTech-Protein-Isolate-Peptides/product-reviews/B01JUA770G/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=1
https://www.amazon.com/MuscleTech-NitroTech-Protein-Isolate-Peptides/product-reviews/B01JUA770G/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=2
https://www.amazon.com/MuscleTech-NitroTech-Protein-Isolate-Peptides/product-reviews/B01JUA770G/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=3
https://www.amazon.com/MuscleTech-NitroTech-Protein-Isolate-Peptides/product-reviews/B01JUA770G/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_

In [20]:
len(reviews)

8741

In [21]:
len(link_per_comment)

8741

In [22]:
len(rating_score)

8741

In [23]:
len(rating_date)

8741

In [24]:
rating_score

['5.0 out of 5 stars',
 '5.0 out of 5 stars',
 '5.0 out of 5 stars',
 '5.0 out of 5 stars',
 '4.0 out of 5 stars',
 '4.0 out of 5 stars',
 '4.0 out of 5 stars',
 '4.0 out of 5 stars',
 '4.0 out of 5 stars',
 '4.0 out of 5 stars',
 '4.0 out of 5 stars',
 '4.0 out of 5 stars',
 '4.0 out of 5 stars',
 '4.0 out of 5 stars',
 '4.0 out of 5 stars',
 '4.0 out of 5 stars',
 '4.0 out of 5 stars',
 '4.0 out of 5 stars',
 '5.0 out of 5 stars',
 '5.0 out of 5 stars',
 '5.0 out of 5 stars',
 '3.0 out of 5 stars',
 '3.0 out of 5 stars',
 '5.0 out of 5 stars',
 '5.0 out of 5 stars',
 '5.0 out of 5 stars',
 '5.0 out of 5 stars',
 '5.0 out of 5 stars',
 '5.0 out of 5 stars',
 '5.0 out of 5 stars',
 '5.0 out of 5 stars',
 '5.0 out of 5 stars',
 '5.0 out of 5 stars',
 '5.0 out of 5 stars',
 '5.0 out of 5 stars',
 '5.0 out of 5 stars',
 '5.0 out of 5 stars',
 '5.0 out of 5 stars',
 '5.0 out of 5 stars',
 '3.0 out of 5 stars',
 '5.0 out of 5 stars',
 '5.0 out of 5 stars',
 '3.0 out of 5 stars',
 '1.0 out o

In [25]:
rating_date

['Reviewed in the United States 🇺🇸 on January 25, 2023',
 'Reviewed in the United States 🇺🇸 on February 6, 2023',
 'Reviewed in the United States 🇺🇸 on January 27, 2023',
 'Reviewed in the United States 🇺🇸 on January 27, 2023',
 'Reviewed in the United States 🇺🇸 on February 18, 2023',
 'Reviewed in the United States 🇺🇸 on February 16, 2023',
 'Reviewed in the United States 🇺🇸 on February 19, 2023',
 'Reviewed in the United States 🇺🇸 on February 9, 2023',
 'Reviewed in the United States 🇺🇸 on February 16, 2023',
 'Reviewed in the United States 🇺🇸 on February 8, 2023',
 'Reviewed in the United States 🇺🇸 on February 5, 2023',
 'Reviewed in the United States 🇺🇸 on February 5, 2023',
 'Reviewed in the United States 🇺🇸 on January 31, 2023',
 'Reviewed in the United States 🇺🇸 on February 2, 2023',
 'Reviewed in the United States 🇺🇸 on January 31, 2023',
 'Reviewed in the United States 🇺🇸 on February 8, 2023',
 'Reviewed in the United States 🇺🇸 on February 6, 2023',
 'Reviewed in the United St

In [26]:
product_name= []
for i in range(len(link_per_comment)):
    name = re.search(r'https://www.amazon.com/(.*?)/product-reviews/', link_per_comment[i]).group(1)
    product_name.append(name)

In [27]:
product_name

['NatureWorks-HydroMATE-Electrolytes-Chocolate-Count',
 'NatureWorks-HydroMATE-Electrolytes-Chocolate-Count',
 'NatureWorks-HydroMATE-Electrolytes-Chocolate-Count',
 'NatureWorks-HydroMATE-Electrolytes-Chocolate-Count',
 'NatureWorks-HydroMATE-Electrolytes-Chocolate-Count',
 'NatureWorks-HydroMATE-Electrolytes-Chocolate-Count',
 'NatureWorks-HydroMATE-Electrolytes-Chocolate-Count',
 'NatureWorks-HydroMATE-Electrolytes-Chocolate-Count',
 'NatureWorks-HydroMATE-Electrolytes-Chocolate-Count',
 'NatureWorks-HydroMATE-Electrolytes-Chocolate-Count',
 'NatureWorks-HydroMATE-Electrolytes-Chocolate-Count',
 'NatureWorks-HydroMATE-Electrolytes-Chocolate-Count',
 'NatureWorks-HydroMATE-Electrolytes-Chocolate-Count',
 'NatureWorks-HydroMATE-Electrolytes-Chocolate-Count',
 'NatureWorks-HydroMATE-Electrolytes-Chocolate-Count',
 'NatureWorks-HydroMATE-Electrolytes-Chocolate-Count',
 'NatureWorks-HydroMATE-Electrolytes-Chocolate-Count',
 'NatureWorks-HydroMATE-Electrolytes-Chocolate-Count',
 'NatureWo

In [28]:
cleaned_rating= []
for i in range(len(rating_score)):
    decimal_score= re.search(r'(.*?) out of', rating_score[i]).group(1)
    cleaned_rating.append(float(decimal_score))

In [29]:
cleaned_rating

[5.0,
 5.0,
 5.0,
 5.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 5.0,
 5.0,
 5.0,
 3.0,
 3.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 3.0,
 5.0,
 5.0,
 3.0,
 1.0,
 5.0,
 3.0,
 5.0,
 5.0,
 4.0,
 5.0,
 2.0,
 5.0,
 5.0,
 5.0,
 4.0,
 5.0,
 3.0,
 5.0,
 4.0,
 5.0,
 5.0,
 5.0,
 5.0,
 2.0,
 2.0,
 2.0,
 5.0,
 1.0,
 1.0,
 1.0,
 2.0,
 1.0,
 5.0,
 3.0,
 1.0,
 4.0,
 5.0,
 4.0,
 4.0,
 4.0,
 3.0,
 4.0,
 5.0,
 5.0,
 5.0,
 5.0,
 2.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 1.0,
 1.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 4.0,
 5.0,
 4.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 4.0,
 4.0,
 4.0,
 5.0,
 5.0,
 4.0,
 5.0,
 5.0,
 4.0,
 4.0,
 5.0,
 5.0,
 5.0,
 5.0,
 4.0,
 4.0,
 5.0,
 4.0,
 4.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 4.0,
 5.0,
 5.0,
 5.0,
 4.0,
 5.0,
 4.0,
 5.0,
 4.0,
 4.0,
 4.0,
 5.0,
 5.0,
 5.0,
 4.0,
 4.0,
 5.0,
 4.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0

In [30]:
cleaned_date= []
for i in range(len(rating_date)):
    pattern= re.compile("(Jan(uary)?|Feb(ruary)?|Mar(ch)?|Apr(il)?|May|Jun(e)?|"
    "Jul(y)?|Aug(ust)?|Sep(tember)?|Oct(ober)?|Nov(ember)?|"
    "Dec(ember)?)\s+\d{1,2},\s+\d{4}")
    new_date= pattern.search(rating_date[i]).group()
    new_date2= datetime.strptime(new_date, '%B %d, %Y')
    new_date3= new_date2.strftime('%Y-%m-%d')
    cleaned_date.append(new_date3)

In [31]:
cleaned_date

['2023-01-25',
 '2023-02-06',
 '2023-01-27',
 '2023-01-27',
 '2023-02-18',
 '2023-02-16',
 '2023-02-19',
 '2023-02-09',
 '2023-02-16',
 '2023-02-08',
 '2023-02-05',
 '2023-02-05',
 '2023-01-31',
 '2023-02-02',
 '2023-01-31',
 '2023-02-08',
 '2023-02-06',
 '2023-02-03',
 '2023-02-02',
 '2023-02-14',
 '2023-02-03',
 '2023-02-02',
 '2023-02-02',
 '2023-02-05',
 '2023-02-13',
 '2023-02-19',
 '2023-02-20',
 '2023-02-04',
 '2023-02-09',
 '2023-02-04',
 '2023-02-13',
 '2023-02-07',
 '2023-02-13',
 '2023-02-05',
 '2023-02-13',
 '2023-02-02',
 '2023-02-02',
 '2023-02-15',
 '2023-02-02',
 '2023-01-31',
 '2023-02-15',
 '2023-02-04',
 '2023-02-10',
 '2023-02-18',
 '2023-02-14',
 '2022-11-16',
 '2022-07-04',
 '2022-04-12',
 '2021-02-20',
 '2020-01-04',
 '2022-04-12',
 '2021-08-07',
 '2020-03-28',
 '2021-07-09',
 '2019-02-11',
 '2019-09-11',
 '2021-07-31',
 '2021-06-04',
 '2019-03-05',
 '2020-10-31',
 '2020-02-21',
 '2019-04-09',
 '2019-12-09',
 '2021-07-02',
 '2021-05-03',
 '2020-05-13',
 '2019-01-

## Final step - create data frame consisting of a table with headers: product name, date, rating score (5 being the highest), reviews, and respective link. At the end, import scraped data into CSV file which is later used for our sentiment analysis.

In [32]:
rev= {'Product_Name':product_name, 'Date':cleaned_date, 'Rating_Score':cleaned_rating, 'Reviews':reviews, 'Link':link_per_comment}

In [33]:
review_data= pd.DataFrame.from_dict(rev)
pd.set_option('max_colwidth',5200)

In [34]:
review_data.head(40)

,Product_Name,Date,Rating_Score,Reviews,Link
0,NatureWorks-HydroMATE-Electrolytes-Chocolate-Count,2023-01-25,5.0,"\nI love this. I make it for myself and my kids as hot chocolate or “chocolate milk” and the taste is so good they have no idea it’s good for them. Other brands taste like medicine, this is just delicious. Always feel refreshed and energized afterwards which you cannot say for most hydration and electrolyte drinks! Will definitely keep buying it!!!\n",https://www.amazon.com/NatureWorks-HydroMATE-Electrolytes-Chocolate-Count/product-reviews/B0BRT77ZK8/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=1
1,NatureWorks-HydroMATE-Electrolytes-Chocolate-Count,2023-02-06,5.0,\nTakes away lightheadedness and makes my husband feel much better..He's 71 and takes this willingly everyday because he knows it helps him.\n,https://www.amazon.com/NatureWorks-HydroMATE-Electrolytes-Chocolate-Count/product-reviews/B0BRT77ZK8/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=1
2,NatureWorks-HydroMATE-Electrolytes-Chocolate-Count,2023-01-27,5.0,\nThe chocolate tastes delicious! I drink it every morning and it energizes me for my day. I will definitely be buying again. 10/10 would recommend.\n,https://www.amazon.com/NatureWorks-HydroMATE-Electrolytes-Chocolate-Count/product-reviews/B0BRT77ZK8/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=1
3,NatureWorks-HydroMATE-Electrolytes-Chocolate-Count,2023-01-27,5.0,\nI absolutely love this! My buddy gave me a few of his and immediately I was sold! Taste is pretty good also!!!\n,https://www.amazon.com/NatureWorks-HydroMATE-Electrolytes-Chocolate-Count/product-reviews/B0BRT77ZK8/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=1
4,NatureWorks-HydroMATE-Electrolytes-Chocolate-Count,2023-02-18,4.0,"\nI like to work out regularly. This includes weight-lifting on some days and running on the treadmill for other days. Given how depleting it can be to run for prolonged periods of time, I like to have an electrolyte drink after my cardio days on the treadmill to try to restore my electrolytes. After having tried this electrolyte drink mix, I must say that I am rather impressed.This chocolate flavored hydration mix actually tastes better than I thought it would. I was a little bit hesitant as I have normally only had fruity-flavored electrolyte drink mixes before, and nothing I have had was similar to the rich flavor of chocolate. However, after having a mix with 8 ounces of water, I was surprised to note that it tasted like hot chocolate - although cold, because I did not use hot water. At 8 ounces of water, it did taste a bit watered down still. I believe that this drink will be most enjoyable if used with hot water or milk for a richer taste, and potentially with less water than they recommend (they recommend 8 to 16 ounces of water per stick).There are a couple of flaws that I have with this product. Firstly, the drink mix did not fully dissolve well. Some other electrolyte drink mixes that I have used have dissolved better than this one. Also, for this drink mix to be most enjoyable, I believe one should use milk or hot water (or hot milk, for that matter). This is an extra step versus with fruity flavors that can be very enjoyable in cold water (from the sink for me). As I enjoy convenience and wouldn't want to heat water each time before use, this is a flaw for me.I like how these sticks have a good vitamin profile. Each stick has significant amount of Vitamin C, B6, and B12, as well as some zinc. You're getting more than just electrolytes with these sticks. There are 2.5 grams of added sugar per stick, which is higher than I typically like as I try to avoid added sugar. But, there are only 10 calories per stick, so if you're a calorie counter, these might be for you.Given the rather affordable price of these electrolyte sticks ($36 for 30 sticks at the time of writing), I think that these could certainly 

In [35]:
review_data.shape

(8741, 5)

In [36]:
# Converting the dataframe to a csv file (raw data)
review_data.to_csv('Whey_Protein_Amazon_Scraped_Reviews_raw.csv')

In [37]:
#Remove duplicates created by the product's own favor variation. For example: Muscle Milk's vanilla vs. chocolate
df = review_data.drop_duplicates(subset=["Reviews"], keep='first')

In [38]:
df.shape

(3314, 5)

In [39]:
#converting the dataframe to a csv file so as to use it later for further analysis (no duplicated comments)
df.to_csv('Whey_Protein_Amazon_Scraped_Reviews.csv') 